In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Jovian Commit Essentials
# Please retain and execute this cell without modifying the contents for `jovian.commit` to work
!pip install jovian --upgrade -q
import jovian
jovian.utils.colab.set_colab_file_id('17dyfBot99KhbR1yp5KyVdlqkO7uG48Xa')

In [ ]:
# Imports
import torch
import torchvision
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split

In [ ]:
df= pd.read_csv('../input/facial-expression-recognitionferchallenge/fer2013/fer2013/fer2013.csv')

In [ ]:
df.shape

In [ ]:
df.columns


In [ ]:
df.sample(5)

In [ ]:
df['emotion'].unique()

In [ ]:
print("There are total of  {} different emotions in our dataset".format(len(set(df['emotion']))))

In [ ]:
df.dtypes

In [ ]:
df.info()

In [ ]:

## what are the total number of pixels
import math
(len(df['pixels'][0].split()))

In [ ]:
## check the image matrix shape to be transformed to
np.sqrt((len(df['pixels'][0].split())))

In [ ]:
# df['pixels'] = df['pixels'].str.split()

In [ ]:
# df.sample()

In [ ]:
# df['pixels'].apply(pd.to_numeric)

In [ ]:
torch.cuda.is_available()

In [ ]:
# (np.array(df['pixels'][0].split(' ')).reshape(1,48,48).astype('float32')/255).shape

In [ ]:
# df['image_data']= df['pixels'].apply(lambda x: pd.Series(x.split(' ')).astype('float32').values.reshape(1,48,48))

In [ ]:
# df3 = df.copy()
# df_melt = df3.assign(pixels = df.pixels.str.split(" "))
# df_final = df_melt.pixels.apply(pd.Series).merge(df_melt,right_index=True,left_index=True)
# df = df_final.drop(columns=['pixels','image_data'],axis=1)

In [ ]:
# df.to_csv('ImageDataset.csv')

In [ ]:
df = pd.read_csv('../input/fer-dataset/ImageDataset.csv')

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
set(df['Usage'])

In [ ]:
df_train = df[df['Usage']=='Training']
df_valid = df[df['Usage']=='PrivateTest']
df_test = df[df['Usage']=='PublicTest']

In [ ]:
print(df_train.shape,df_valid.shape,df_test.shape)

In [ ]:
df_train.sample(1)

In [ ]:
train_input = df_train.drop(columns= ['Unnamed: 0','emotion','Usage'],axis=1).values.reshape(-1,1,48,48).astype('float32')
# train_input = df_train.drop(columns= ['Unnamed: 0','emotion','Usage'],axis=1).values

In [ ]:
train_input[0]

In [ ]:
plt.imshow(train_input[0].squeeze(),cmap='gray')

In [ ]:
train_output= df_train['emotion'].values
print(train_output.shape)
train_output


In [ ]:
dataset_train = TensorDataset(torch.from_numpy(train_input.astype('float32')),torch.tensor(train_output))

In [ ]:
len(dataset_train)

In [ ]:
dataset_train[0]

## similarly we will generate pytorch datasets for our validation and test sets

In [ ]:
valid_input = df_valid.drop(columns= ['Unnamed: 0','emotion','Usage'],axis=1).values.reshape(-1,1,48,48).astype('float32')
valid_output= df_valid['emotion'].values

test_input = df_test.drop(columns= ['Unnamed: 0','emotion','Usage'],axis=1).values.reshape(-1,1,48,48).astype('float32')
test_output= df_test['emotion'].values


dataset_valid = TensorDataset(torch.from_numpy(valid_input.astype('float32')),torch.tensor(valid_output))
dataset_test = TensorDataset(torch.from_numpy(test_input.astype('float32')),torch.tensor(test_output))




In [ ]:
print(len(dataset_train))
print(len(dataset_valid))
print(len(dataset_test))

In [ ]:
dataset_train[0]

In [ ]:
dataset_valid[0]

In [ ]:
dataset_test[0]

## Dataloaders

In [ ]:
batch_size = 256

train_loader = DataLoader(dataset_train, batch_size, shuffle=True)
val_loader = DataLoader(dataset_valid, batch_size*2)
test_loader = DataLoader(dataset_test, batch_size*2)

In [ ]:
for images,labels in train_loader:
    print(images.shape)
    print(labels)
    break

In [ ]:
labels.shape

## Using a GPU

In [ ]:
## Moving all the data to the cuda/GPU

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

In [ ]:
device = get_default_device()
device

In [ ]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [ ]:
for images, labels in train_loader:
    print(images.shape)
    images = to_device(images, device)
    print(images.device)
    break

In [ ]:
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
train_loader = DeviceDataLoader(train_loader, device)
val_loader = DeviceDataLoader(val_loader, device)

In [ ]:
for xb, yb in val_loader:
    print('xb.device:', xb.device)
    print('yb:', yb)
    break

## Model

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [ ]:
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))

In [ ]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []

    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        

        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [ ]:
class FER(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12 * 9 * 9, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=7)
        
    def forward(self, xb):
         # (1) input layer
        t = xb

        # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # (4) hidden linear layer
        t = t.reshape(-1, 12 * 9 * 9)
        t = self.fc1(t)
        t = F.relu(t)

        # (5) hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)

        # (6) output layer
        t = self.out(t)
        #t = F.softmax(t, dim=1)

        return t

In [ ]:
## Creating an instance of the model and Moving model to the GPU
## model = FER()
model = to_device(FER(), device)

Before you train the model, it's a good idea to check the validation loss & accuracy with the initial set of weights.

In [ ]:
history = [evaluate(model, val_loader)]
history

In [ ]:
# print(model.conv1.weight.shape, model.linear.bias.shape)
list(model.parameters())

In [ ]:
for images, labels in train_loader:
    print(images.shape)
    outputs = model(images)
    break

print('outputs.shape : ', outputs.shape)
print('Sample outputs :\n', outputs[:2].data)

### Training a model

In [ ]:
history += fit(50, 0.001, model, train_loader, val_loader)

In [ ]:
# history += fit(2, 0.05, model, train_loader, val_loader)

In [ ]:
# history += fit(5, 0.005, model, train_loader, val_loader)

### Plot

In [ ]:
def plot_losses(history):
    losses = [x['val_loss'] for x in history]
    plt.plot(losses, '-x')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.title('Loss vs. No. of epochs');

In [ ]:
def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs');

In [ ]:
plot_losses(history)

In [ ]:
plot_accuracies(history)

In [ ]:
model


In [ ]:
## let's check how the model is performing on the test set

In [ ]:
test_loader = DeviceDataLoader(test_loader, device)

In [ ]:
evaluate(model, test_loader)

In [ ]:
## prediction on samples

def predict_image(img, model):
    xb = to_device(img.unsqueeze(0), device)
    yb = model(xb)
    _, preds  = torch.max(yb, dim=1)
    return preds[0].item()

In [ ]:
img, label = dataset_test[0]

In [ ]:
predict_image(img,model)

In [ ]:
label

In [ ]:
plt.imshow(img.squeeze(), cmap='gray')